In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a7196fb40227eba950dcb7e052a602cba013adfd12350ed9ca0fc526e6fefeee
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Getting started with Spark: Spark SQL in Python

This tutorial is based on [Spark SQL Guide - Getting started](https://spark.apache.org/docs/latest/sql-getting-started.html).

For this demo we used the city of Vienna trees dataset ("Baumkataster") made available by [Open Data Österreich](https://www.data.gv.at) and downloadable from [here](https://www.data.gv.at/katalog/dataset/c91a4635-8b7d-43fe-9b27-d95dec8392a7) .

# Table of contents
1. [Spark session](#sparkSession)
2. [Spark configuration](#sparkConfiguration)

## Spark session <a name="sparkSession"></a>

We're going to start by creating a Spark _session_. Our Spark job will be named "Python Spark SQL basic example". `spark` is the variable holding our Spark session.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

Read the file into a Spark [_dataframe_](https://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes).

In [4]:
df = spark.read \
          .load("FME_BaumdatenBearbeitet_OGD_20190205.csv",
           format="csv", sep=";", header="true", encoding="iso-8859-1")

**Note:** we assume that the file `FME_BaumdatenBearbeitet_OGD_20190205.csv` is in your local directory. If at this point you get an error message that looks like `AnalysisException: 'Path does not exist` then check your [Spark configuration](#sparkConfig) for how to define the correct file path.

Show first three lines of Spark dataframe

In [5]:
df.show(3)

+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+----------------+----------------+
|Flaeche|BaumNr|     Gattung|       Art|Sorte|NameDeutsch|Hoehe|Schirmdurchmesser|Stammumfang|Typ| XPos|  YPos|             lon|             lat|
+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+----------------+----------------+
|      0|     0|   ZumTesten|         0|    0|   20190205|    0|                0|          0|  0|70000|350000|14,2757549011314|48,2844031941042|
|    870|  021a|     Quercus|       sp.|    -|      Eiche|    6|                3|         31|  L|72431|354949|14,3093549528352|48,3286271802142|
|    572|   127|Liriodendron|tulipifera|    -| Tulpenbaum|    5|                2|         21|  L|71171|353742|14,2921648325343|48,3179178510249|
+-------+------+------------+----------+-----+-----------+-----+-----------------+-----------+---+-----+------+-------------

For pretty-printing you can use `toPandas()`

In [6]:
df.toPandas().head(3)

,Flaeche,BaumNr,Gattung,Art,Sorte,NameDeutsch,Hoehe,Schirmdurchmesser,Stammumfang,Typ,XPos,YPos,lon,lat
0,0,0,ZumTesten,0,0,20190205,0,0,0,0,70000,350000,"14,2757549011314","48,2844031941042"
1,870,021a,Quercus,sp.,-,Eiche,6,3,31,L,72431,354949,"14,3093549528352","48,3286271802142"
2,572,127,Liriodendron,tulipifera,-,Tulpenbaum,5,2,21,L,71171,353742,"14,2921648325343","48,3179178510249"


Show number of different trees (count German names in `df` and sort by count)

In [7]:
df.groupBy("NameDeutsch").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|         NameDeutsch|count|
+--------------------+-----+
|        Winter-Linde| 1583|
|          Weiß-Birke| 1442|
|         Spitz-Ahorn| 1273|
|         Stiel-Eiche| 1228|
|               Ahorn| 1079|
|           Hainbuche| 1036|
|       Gemeine Esche|  987|
|Ahornblättrige-Pl...|  961|
|            Rotbuche|  747|
|               Linde|  688|
|          Feld-Ahorn|  637|
|          Berg-Ahorn|  566|
|    Säulen-Hainbuche|  507|
|Gemeine Rosskastanie|  461|
|          Tulpenbaum|  436|
|             Robinie|  358|
|        Silber-Weide|  353|
|      Schwarz-Kiefer|  321|
|           Baumhasel|  258|
|    Serbische Fichte|  257|
+--------------------+-----+
only showing top 20 rows



An example of SQL query (see [Running SQL Queries Programmatically](https://spark.apache.org/docs/latest/sql-getting-started.html#running-sql-queries-programmatically)): let's sort trees by height ("Hoehe").

In [8]:
df.createOrReplaceTempView("baeume")

In [9]:
spark.sql("SELECT BaumNr, NameDeutsch, Hoehe, lat, lon FROM baeume order  by Hoehe desc").show()

+------+--------------------+-----+----------------+----------------+
|BaumNr|         NameDeutsch|Hoehe|             lat|             lon|
+------+--------------------+-----+----------------+----------------+
|   129|          Weiß-Birke|   99|48,2764221420506|14,3029622419918|
|   005|           Hainbuche|   99|48,3107341721266|14,2818381176027|
|   844|            Weißdorn|   99|48,3198141692826|14,3032456456049|
|   037|Gemeine Kiefer / ...|   95|48,3021818905511|14,2769106844535|
|   007|          Tulpenbaum|   90|48,2765758955147|14,3027882173276|
|   051|          Weiß-Birke|    9|48,2671468323820|14,2832242240464|
|   009|        Sommer-Linde|    9|48,2383321988402|14,3668869798697|
|   001|    Säulen-Hainbuche|    9|48,2973425597313|14,3138560746343|
|   025|       Schwarz-Birke|    9|48,2749605071833|14,2781176169409|
|   004|            Rotbuche|    9|48,2768058741599|14,2821412254345|
|   001|        Silber-Ahorn|    9|48,2777290002478|14,2797511856080|
|   011|            

The height data doesn't seem to be up-to-date.

## Spark configuration <a name="sparkConfiguration"></a>

Spark properties control most application settings and are configured separately for each application. These properties can be set directly on a `SparkConf` passed to your `SparkContext` (from [Apache Spark documentation](https://spark.apache.org/docs/latest/configuration.html#spark-properties)).  

We've already seen how to modify the `SparkConf` when we created our Spark application session with the command:
<pre>
    spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()
</pre>

Let us look at the rest of the Spark configuration.

In [10]:
from pyspark.conf import SparkConf
spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'Python Spark SQL basic example'),
 ('spark.sql.w

The property `spark.app.name` is the name of our app that we just defined.

Another important property is `spark.master`. This defines the _master URL_  for the Spark application. A list of all admissible values for `spark.master` is given here: [master-urls](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls).

In this example the Spark master URL is `local[*]`, this means that our Spark application will run locally with as many worker threads as logical cores on our local machine.

If you have a Hadoop cluster available you can deploy your Spark application on Yarn by setting the option `spark.master = yarn`. Let's do that and then check the Spark configuration once again.

In [11]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master('yarn') \
    .getOrCreate()

spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'Python Spark SQL basic example'),
 ('spark.sql.w

With this configuration our Spark application will run on the Hadoop cluster and its resources will be managed by Yarn.

**Note:** If the Hadoop cluster is configured with HDFS as its default filesystem, then you need to upload your CSV file to Hadoop in order to be able to read it:
<code>
    hdfs dfs -put FME_BaumdatenBearbeitet_OGD_20190205.csv FME_BaumdatenBearbeitet_OGD_20190205.csv
</code>
and then you can just use `.load( ...) ` again.